# Elastic Client

In [1]:
from ltr.client import OpenSearchClient
client = OpenSearchClient()

http://localhost:9201/_ltr; <OpenSearch([{'host': 'localhost', 'port': 9201}])>


# Download & Build Index (run once)

If you don't already have the downloaded dependencies; if you don't have TheMovieDB data indexed run this

In [2]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'
download([corpus, judgments], dest='data/');

data/tmdb.json already exists
data/title_judgments.txt already exists


In [3]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

Index tmdb already exists. Use `force = True` to delete and recreate


## Features for movie titles

We'll be searching movie titles (think searching for a specific movie on Netflix). And we have a set of judgments around the appropriatte movie to return. IE search for "Star Wars" return good star wars matches, in quality order...

These cover various aspects of the problem (searching title by phrase, title bm25 score, release date, etc). We'll use this to explore and analyze a simple model

In [4]:
client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
            { #1
                "name": "title_has_phrase",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            { #2
                "name": "title_has_terms",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            { #3
                "name": "title_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"title": "{{keywords}}"}
                }
            },
            { #4
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"overview": "{{keywords}}"}
                }
            },
            { #5
                "name": "overview_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"overview": "{{keywords}}"}
                }
            },
            { #6
                "name": "title_fuzzy",
                "params": ["keywords"],
                "template": {
                    "match": {"title": 
                                {"query": "{{keywords}}",
                                 "fuzziness": "AUTO"}}
                }
            },
             { #7
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            }
            
            
            ]
    }}


client.create_featureset(index='tmdb', name='title', ftr_config=config)

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create title feature set [Status: 201]


## Training Set Generation

Log out features for each of the above queries out to a training set file

In [5]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='title')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))

Recognizing 40 queries


## Feature Search: which features work best?

What combination of these features work best? Train a model with every combination, and use k-fold cross valudation (see `kcv=15` below). The combination with the best NDCG is output

In [6]:
from ltr.ranklib import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               training_set=ftr_logger.logged,
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,5,7],
                                               featureSet='title')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1] TEST NDCG@10=0.9071
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2] TEST NDCG@10=0.9067
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs

## Compare to model w/ all features

Compare the features output above (something like...)

```
Impact of each feature on the model
3 - 17728.651362443747
4 - 10986.88505569509
7 - 7504.573843344015
1 - 1580.321225819952
```

to one trained with the full model. Notice how features have different impacts. This is due to feature dependency

In [7]:
from ltr.ranklib import train
trainResponse  = train(client,
                  training_set=ftr_logger.logged,
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  features=[1,2,3,4,5,6,7],
                  index='tmdb',
                  featureSet='title',
                  modelName='title')

trainLog = trainResponse.trainingLogs[0]

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/title_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt
Delete model title: 404
Created Model title [Status: 201]
Model saved

Impact of each feature on the model
title_fuzzy - 18766.11246284367
title_bm25 - 14740.291519685004
overview_bm25 - 10002.493638661677
release_year - 6315.7831389888
overview_phrase_bm25 - 2898.441699180107
title_has_phrase - 2059.7002349446443
title_has_terms - 0.009750295980646107
0.9129
0.9159
0.917
0.9198
0.9198
0.9227
0.926
0.926
0.926
0.9259
0.9316
0.9348
0.934
0.9338
0.9349
0.9349
0.9346
0.9392
0.9392
0.9394
Train NDCG@10 0.9394


## Bias towards fewer features

By adding a 'cost', to feature search, we add a multiplier that punishes models with more features slightly. This results in a tiny bias towards simpler models all things being equal. As we'd prefer one that doesn't need to execute more features

In [8]:
from ltr.ranklib import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               training_set=ftr_logger.logged,
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               featureCost=0.02,# 1.0-cost ^ num_features
                                               features=[1,2,7],
                                               featureSet='title')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1] TEST NDCG@10=0.9071 after cost 0.9071
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2] TEST NDCG@10=0.9067 after cost 0.9067
/var/folde

# Evaluating the Model

It's interesting to see what features our model makes use of, but we need guidance on adding additional features to the model. We know our model is an ensemble of decision trees. Wouldn't it be cool if we could trace where documents end up on that decision tree?

Specifically, we care about problems. Or what we will call affectionately *whoopsies*. 

As a 'whoopsie' example, consider the query "Rambo". if a '0' document like 'First Daughter' ranked the same or higher than a '4' document ("Rambo")., that's a problem. It's also an opportunity for improvement. We'd want to isolate that, see if it's indicative of a broader trend, and thus worth adding a feature for.

Let's see a concrete example

In [9]:
from ltr.MART_model import eval_model
from itertools import groupby

features, _ = client.feature_set(index='tmdb', name='title')

for qid, query_judgments in groupby(ftr_logger.logged, key=lambda j: j.qid):
    print(qid)
    if qid == 1: # Rambo
        model = eval_model(modelName='title',
                               features=features,
                               judgments=query_judgments)

        print()
        print("## Evaluating graded docs for search keywords 'Rambo'")
        print()
        print(model)
        break

Fetched FeatureSet title [Status: 200]
1
Whoopsies on LAMBDAMart
Header At 6

## Evaluating graded docs for search keywords 'Rambo'

if title_bm25 > 10.6643305:
  if title_has_phrase > 0.0:
    if overview_phrase_bm25 > 8.602641:
      if release_year > 2014.0:
        <= 0.2000(0/0/)
      else:
        if overview_phrase_bm25 > 12.821273:
          <= 0.1887(0/0/)
        else:
          <= 0.2000(1/0/)
    else:
      if overview_bm25 > 0.0:
        if title_fuzzy > 11.08591:
          if title_bm25 > 11.566039:
            if title_bm25 > 17.490765:
              <= 0.2000(0/0/)
            else:
              if overview_bm25 > 8.614531:
                <= 0.1995(0/0/)
              else:
                <= 0.2000(0/0/)
          else:
            <= -0.2000(0/0/)
        else:
          <= 0.2000(0/0/)
      else:
        if title_bm25 > 13.815247:
          if title_fuzzy > 13.919913:
            <= 0.1989(0/0/)
          else:
            <= 0.2000(0/0/)
        else:
         

## Examining our evaluation for whoopsies

Let's looks at one tree in our ensemble, te see how it was evaluated.

```
if title_bm25 > 10.664251:
  if title_phrase > 0.0:
    if title_bm25 > 13.815164:
      if release_year > 2000.0:
        <= 0.1215(0/0/)
      else:
        <= 0.1240(0/0/)
    else:
      if title_bm25 > 10.667803:
        if overview_bm25 > 0.0:
          <= 0.1194(0/0/)
        else:
          <= 0.1161(1/0/)
      else:
        <= 0.1264(0/0/)
  else:
    <= 0.0800(0/0/)
else:
  if title_phrase > 0.0:
    if title_bm25 > 8.115499:
      if title_bm25 > 8.217656:
        <= 0.1097(2/1/qid:40:2(12180)-3(140607))
      else:
        <= 0.1559(0/0/)
    else:
      <= -0.0021(2/1/qid:40:2(1895)-3(330459))
  else:
    <= -0.1093(25/1/qid:40:0(85783)-3(1892))
```

You'll notice here this tree is represented by a series of if statements, where the feature's name is used. This is handy as it lets us take apart the structure of the tree.

You'll also notice the leaf nodes starting with 

```
<=
```

These leaf nodes have a floating point value, corresponding to the relevance score that documents ending up here will have. Each leaf also has three items in paranthesis, such as `(2/1/qid:40:2(1895)-3(330459))`. This is a report summarizing the result of evaluating the tree on the provided judgment list. Indicating:


```



   +--- 2 Documents evaluated to this leaf node                   +-- max grade doc eval'd to this leaf
   |                                                              |
   | +----- 1 'whoopsie' occured                                  |  +-- corresp. doc id of max doc
   | |                                                            |  |
   | |   +--- details on each whoopsie ----------- qid:40:2(1985)-3(330459)
   | |   |                                              | |  |
  (2/1/qid:40:2(1895)-3(330459))                        | |  |
                                                        | |  + doc id of min graded doc
                                                        | |
                                                        | + min grade of docs eval'd to this leaf
                                                        |
                                                        + query id of whoopsie from judgments
```


Looking at Star Wars, our biggest issues in this tree are with the bottom-most leaf. Here

```
if title_bm25 > 10.664251:
  ...
else:
  if title_phrase > 0.0:
    ...
  else:
    <= -0.1093(25/1/qid:40:0(85783)-3(1892))
```


Document 85783 (a '0') and doc 1892 are given the same grade.

### Whoopsie, from the query perspective

Whoopsies can also be examined at the "query" level to see for a query id, how many whoopsies existed, and what was the evaluation for that query at each tree. This can help see if an error was fixed later in the ensemble of trees.

In [10]:
whoopsies = model.whoopsies()
for qid, whoopsie in whoopsies.items():
    print("== QID %s ==" % qid)
    print("%s - %s" % (whoopsie.count, whoopsie.totalMagnitude))
    print(whoopsie.perTreeReport())

== QID 1 ==
20 - 56
tree:0=>0(319074)-4(1368);tree:1=>0(319074)-4(1368);tree:2=>0(319074)-4(1368);tree:3=>0(319074)-4(1368);tree:4=>0(319074)-4(1368);tree:5=>0(319074)-2(13258);tree:6=>0(319074)-2(13258);tree:7=>0(319074)-2(13258);tree:8=>0(319074)-2(13258);tree:9=>0(319074)-2(13258);tree:10=>0(319074)-1(208982);tree:11=>1(31362)-4(1368);tree:12=>1(31362)-4(1368);tree:13=>1(31362)-4(1368);tree:14=>1(31362)-4(1368);tree:15=>1(31362)-4(1368);tree:16=>1(31362)-4(1368);tree:17=>0(319074)-1(61410);tree:18=>1(31362)-4(1368);tree:19=>1(31362)-4(1368)


In [11]:
### Doc in index... (notice nothing mentions 'star wars')

client.get_doc(index='tmdb', doc_id=1368)

{'id': '1368',
 'title': 'First Blood',
 'overview': 'When former Green Beret John Rambo is harassed by local law enforcement and arrested for vagrancy, the Vietnam vet snaps, runs for the hills and rat-a-tat-tats his way into the action-movie hall of fame. Hounded by a relentless sheriff, Rambo employs heavy-handed guerilla tactics to shake the cops off his tail.',
 'tagline': "This time he's fighting for his life.",
 'directors': ['Ted Kotcheff'],
 'cast': 'Sylvester Stallone Richard Crenna Brian Dennehy Bill McKinney Jack Starrett Michael Talbott Chris Mulkey John McLiam Alf Humphreys David Caruso David L. Crowley Don MacKay Charles A. Tamburro David Petersen Craig Huston',
 'genres': ['Action', 'Adventure', 'Thriller', 'War'],
 'release_date': '1982-10-22',
 'release_year': '1982',
 'poster_path': 'https://image.tmdb.org/t/p/w185/bbYNNEGLXrV3lJpHDg7CKaPscCb.jpg',
 'vote_average': 7.1,
 'vote_count': 1121}

## Add a feature: collection name

We have an intuition about our data, there is a field for the movies "collection name". See it here below:

In [12]:
from ltr.helpers.movies import get_movie
get_movie(1368)

{'id': 1368,
 'title': 'First Blood',
 'video': False,
 'mlensId': '2403',
 'vote_average': 7.1,
 'backdrop_path': '/cU3goO0TMNDWxgDwILAARyiblXK.jpg',
 'tagline': "This time he's fighting for his life.",
 'directors': [{'id': 16544,
   'department': 'Directing',
   'credit_id': '52fe42efc3a36847f802e18b',
   'name': 'Ted Kotcheff',
   'profile_path': '/r4rMwZeClwJR2VCgvYsNjLOTXpB.jpg',
   'job': 'Director'}],
 'release_date': '1982-10-22',
 'belongs_to_collection': {'poster_path': '/feGOEOVrOLyjtEnVa88rQLgD3XY.jpg',
  'id': 5039,
  'backdrop_path': '/Yt2ZxbJv2HM842B6FNMr59Vhyb.jpg',
  'name': 'Rambo Collection'},
 'runtime': 93,
 'popularity': 3.06572,
 'status': 'Released',
 'original_language': 'en',
 'cast': [{'order': 0,
   'id': 16483,
   'cast_id': 20,
   'credit_id': '52fe42efc3a36847f802e1fb',
   'name': 'Sylvester Stallone',
   'profile_path': '/gnmwOa46C2TP35N7ARSzboTdx2u.jpg',
   'character': 'John J. Rambo'},
  {'order': 1,
   'id': 16554,
   'cast_id': 21,
   'credit_id': 

We'll add collection name, and reindex.

In [13]:
def add_collection_name(src_movie, base_doc):
    if 'belongs_to_collection' in src_movie and src_movie['belongs_to_collection'] is not None:
        if 'name' in src_movie['belongs_to_collection']:
            base_doc['collection_name'] = src_movie['belongs_to_collection']['name']
    return base_doc

movies=indexable_movies(movies='data/tmdb.json', enrich=add_collection_name)
rebuild(client, index='tmdb', doc_src=movies, force=True)

Deleted index tmdb [Status: 200]
Created index tmdb [Status: 200]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27846/27846 [00:42<00:00, 658.40it/s]


Streaming Bulk index DONE tmdb [Status: 201]


Confirm it's in our doc now...

In [14]:
client.get_doc(index='tmdb', doc_id=1892)

{'id': '1892',
 'title': 'Return of the Jedi',
 'overview': "As Rebel leaders map their strategy for an all-out attack on the Emperor's newer, bigger Death Star. Han Solo remains frozen in the cavernous desert fortress of Jabba the Hutt, the most loathsome outlaw in the universe, who is also keeping Princess Leia as a slave girl. Now a master of the Force, Luke Skywalker rescues his friends, but he cannot become a true Jedi Knight until he wages his own crucial battle against Darth Vader, who has sworn to win Luke over to the dark side of the Force.",
 'tagline': 'The Empire Falls...',
 'directors': ['Richard Marquand'],
 'cast': "Mark Hamill Harrison Ford Carrie Fisher Billy Dee Williams Anthony Daniels David Prowse Kenny Baker Peter Mayhew Frank Oz Ian McDiarmid James Earl Jones Sebastian Shaw Hayden Christensen Alec Guinness Warwick Davis Caroline Blakiston Michael Pennington Kenneth Colley Michael Carter Denis Lawson Tim Rose Dermot Crowley Jeremy Bulloch Femi Taylor Annie Arbogast

Add it to the features, and retrain....

In [15]:
config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
            {
                "name": "title_phrase",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            {
                "name": "title",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match": {"title": "{{keywords}}"}
                        },
                        "boost": 1.0
                    }  
                }
            },
            {
                "name": "title_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"title": "{{keywords}}"}
                }
            },
            {
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"overview": "{{keywords}}"}
                }
            },
            {
                "name": "overview_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"overview": "{{keywords}}"}
                }
            },
            {
                "name": "title_fuzzy",
                "params": ["keywords"],
                "template": {
                    "match": {"title": 
                                {"query": "{{keywords}}",
                                 "fuzziness": "AUTO"}}
                }
            },
             {
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            },
            {
                "name": "coll_name_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"collection_name": 
                                {"query": "{{keywords}}"}}
                }
            },
             {
                "name": "coll_name_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"collection_name": 
                                {"query": "{{keywords}}"}}
                }
            }
            
            
            ]
    }}

client.create_featureset(index='tmdb', name='title2', ftr_config=config)

from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='title2')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))

Create title2 feature set [Status: 201]
Recognizing 40 queries


In [16]:
from ltr.ranklib import feature_search
rankLibResult, ndcgPerFeature = feature_search(client,
                                               training_set=ftr_logger.logged,
                                               metric2t='NDCG@10',
                                               leafs=20,
                                               trees=20,
                                               kcv=15,
                                               features=[1,2,3,4,5,6,7,8,9],
                                               featureSet='title2')

print()
print("Impact of each feature on the model")
trainLogs = rankLibResult.trainingLogs
for ftrId, impact in trainLogs[-1].impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
for roundDcg in trainLogs[-1].rounds:
    print(roundDcg)
    
print("Avg NDCG@10 when feature included:")
for ftrId, ndcg in ndcgPerFeature.items():
    print("%s => %s" % (ftrId, ndcg))
    
print("Avg K-Fold NDCG@10 %s" % rankLibResult.kcvTestAvg)

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1] TEST NDCG@10=0.9071
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2] TEST NDCG@10=0.9067
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs

Trying features [2, 3] TEST NDCG@10=0.8754
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2, 4] TEST NDCG@10=0.5979
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2, 5] TEST NDCG@10=0.84

Trying features [5, 6] TEST NDCG@10=0.773
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [5, 7] TEST NDCG@10=0.4082
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [5, 8] TEST NDCG@10=0.751

Trying features [1, 3, 5] TEST NDCG@10=0.8822
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1, 3, 6] TEST NDCG@10=0.879
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1, 3, 7] TEST NDCG

Trying features [1, 7, 8] TEST NDCG@10=0.8235
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1, 7, 9] TEST NDCG@10=0.8032
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1, 8, 9] TEST NDC

...

Trying features [1, 3, 5, 6, 7, 9] TEST NDCG@10=0.8773
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1, 3, 5, 6, 8, 9] TEST NDCG@10=0.8888
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features 

Trying features [2, 3, 4, 6, 7, 9] TEST NDCG@10=0.8941
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2, 3, 4, 6, 8, 9] TEST NDCG@10=0.8847
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features 

Trying features [3, 4, 5, 7, 8, 9] TEST NDCG@10=0.8767
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [3, 4, 6, 7, 8, 9] TEST NDCG@10=0.8547
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features 

Trying features [1, 2, 3, 5, 7, 8, 9] TEST NDCG@10=0.9009
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [1, 2, 3, 6, 7, 8, 9] TEST NDCG@10=0.9043
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying fea

Trying features [2, 3, 4, 5, 6, 8, 9] TEST NDCG@10=0.8764
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying features [2, 3, 4, 5, 7, 8, 9] TEST NDCG@10=0.8767
/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/temp_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt -kcv 15 
Trying fea

In [17]:
from ltr.ranklib import train
trainResponse  = train(client,
                  training_set=ftr_logger.logged,
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  index='tmdb',
                  features=[1,4,6,7,8],
                  featureSet='title2',
                  modelName='title2')

trainLog = trainResponse.trainingLogs[0]

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/title2_model.txt  -feature /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/features.txt
Delete model title2: 404
Created Model title2 [Status: 400]


RuntimeError: {"error":{"root_cause":[{"type":"illegal_argument_exception","reason":"Stored feature set [title2] does not exist"}],"type":"illegal_argument_exception","reason":"Stored feature set [title2] does not exist"},"status":400}

In [ ]:
from ltr.MART_model import eval_model
from itertools import groupby

features, _ = client.feature_set(index='tmdb', name='title2')

for qid, query_judgments in groupby(ftr_logger.logged, key=lambda j: j.qid):
    print(qid)
    if qid == 1: # Rambo
        model = eval_model(modelName='title2',
                               features=features,
                               judgments=query_judgments)

        print()
        print("## Evaluating graded docs for search keywords 'Rambo'")
        print()
        print(model)
        break